In [1]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats
import matplotlib as mpl
import itertools
import RNA

In [2]:
#load introns and divide by sense

Introns=pd.read_csv('/Volumes/Storage3/Paramecium.FromChunlong/Data/NMD/20140122.Intron.Count.WTandNMD.Mac51.Validated.tab',sep='\t')

Introns=Introns.query('(Scaffold < 167 or Scaffold==556 ) and WT_Spl > WT_Ret+WT_AS')
#rename gene old gene ID
Introns['old_Gene_ID']=Introns['Gene_ID']
del Introns['Gene_ID']
#delete old annotation intron position
del Introns['Intron_position']
Introns_neg=Introns.query("Sense==-1")
Introns_pos=Introns.query("Sense==1")

In [3]:
# load new exons annotation and divide by sense
exons=pd.read_csv('/Users/sgnan/Desktop/Projects/paramecium/Genes/ptetraurelia_mac_51_annotation_v2.0.gff3',sep='\t',skiprows=2,names=['Scaffold','EuGene','feature','Exon_start_Mac51','Exon_end_Mac51','dot','strand','value','Info']).query('feature=="exon"')

ID_gene=re.compile('PTET.51.1.T[0-9]{7}')

exons=exons.assign(
    Scaffold=exons.Scaffold.apply(lambda x: x.split('_')[1]).astype(int),
    Gene_ID=exons.Info.apply(lambda x:  ID_gene.findall(x)[0]),
    Sense=exons.strand.apply(lambda x: -1 if x =='-' else 1)

).query('Scaffold < 167  or Scaffold==556').filter(['Scaffold','Exon_start_Mac51','Exon_end_Mac51','Sense','Gene_ID'])

Exons_neg=exons.query("Sense==-1").filter(['Scaffold','Exon_start_Mac51','Exon_end_Mac51','Gene_ID','Sense'])
Exons_pos=exons.query("Sense==1").filter(['Scaffold','Exon_start_Mac51','Exon_end_Mac51','Gene_ID','Sense'])


In [4]:
#id position exon
Exons_pos=Exons_pos.sort_values('Exon_start_Mac51')
Exons_pos['position']=Exons_pos.groupby('Gene_ID').cumcount()+1

Exons_neg=Exons_neg.sort_values('Exon_start_Mac51',ascending=False)
Exons_neg['position']=Exons_neg.groupby('Gene_ID').cumcount()+1


In [5]:
# attach preovious and following exon to each intron
Introns_neg=Introns_neg.merge(Exons_neg.loc[:, Exons_neg.columns != 'position'].assign(Intron_start_Mac51=Exons_neg.Exon_end_Mac51+1), how='inner',on=['Scaffold','Intron_start_Mac51','Sense']).rename(columns={"Exon_start_Mac51":"nextExon_start_Mac51",
                                                                                      "Exon_end_Mac51":"nextExon_end_Mac51"
                                                                                     }).merge(Exons_neg.assign(Intron_stop_Mac51=Exons_neg.Exon_start_Mac51-1), how='inner',on=['Scaffold','Gene_ID','Intron_stop_Mac51','Sense']).rename(columns={"Exon_start_Mac51":"prevExon_start_Mac51",
                                                                                      "Exon_end_Mac51":"prevExon_end_Mac51",'position':'Intron_position'
                                                                                     })


Introns_pos=Introns_pos.merge(Exons_pos.loc[:, Exons_pos.columns != 'position'].assign(Intron_stop_Mac51=Exons_pos.Exon_start_Mac51-1), how='inner',on=['Scaffold','Intron_stop_Mac51','Sense']).rename(columns={"Exon_start_Mac51":"nextExon_start_Mac51",
                                                                                      "Exon_end_Mac51":"nextExon_end_Mac51"
                                                                                     }
).merge(Exons_pos.assign(Intron_start_Mac51=Exons_pos.Exon_end_Mac51+1), how='inner',on=['Scaffold','Gene_ID','Intron_start_Mac51','Sense']).rename(columns={"Exon_start_Mac51":"prevExon_start_Mac51",
                                                                                      "Exon_end_Mac51":"prevExon_end_Mac51",'position':'Intron_position'
                                                                                     })



In [6]:
# concatenate introns
Introns_info=pd.concat([Introns_neg,Introns_pos],axis=0)

In [7]:
#convert donors/acceptor into numerical
Introns_info=Introns_info.assign(
Donor=Introns_info.Strong_donor.apply(lambda x: 1 if x=='yes' else 0),
Acceptor=Introns_info.Strong_acceptor.apply(lambda x: 1 if x=='yes' else 0))

In [8]:
100*(Introns_info.shape[0])/Introns.shape[0]

98.73982709582945

In [9]:
# tanscrip info
Transcript=exons.groupby(['Gene_ID']).agg({'Exon_start_Mac51':'min','Exon_end_Mac51':'max','Sense':'count'})
Transcript=Transcript.reset_index().rename(columns={'Exon_start_Mac51':'Transcript_start_Mac51','Exon_end_Mac51':'Transcript_end_Mac51','Sense':'Count'})
Transcript=Transcript.assign(
Count=Transcript.Count-1,
Transcript_size=Transcript.Transcript_end_Mac51.values-Transcript.Transcript_start_Mac51.values
)

In [10]:
Introns_info=Introns_info.merge(Transcript)

In [11]:
#load peaks and add center
Peaks=pd.read_csv('/Users/sgnan/Desktop/Projects/paramecium/common_Nucleosome_peaks.bed',sep='\t',names=['Scaffold','start','end'])
Peaks['center']=(Peaks.start.values+Peaks.end.values)/2
#prepare genes to calculate distance form center
Introns_info=Introns_info.assign(
    center=(Introns_info.Intron_start_Mac51.values+Introns_info.Intron_stop_Mac51.values)/2,
    width=1+Introns_info.Intron_stop_Mac51.values-Introns_info.Intron_start_Mac51.values,
    distance_center=np.nan,
    density=np.nan)

In [12]:
#calculate distance from the center of the closest nucleosome
Scaffold_list2=np.unique(Peaks.Scaffold.values)
for scaffold in Scaffold_list2:
    Introns_info[Introns_info.Scaffold==scaffold]=Introns_info[Introns_info.Scaffold==scaffold].assign(
        distance_center=np.asarray(np.min(abs(np.matrix(Introns_info.query('Scaffold=='+str(scaffold)).center.values)-np.matrix(Peaks.query('Scaffold=='+str(scaffold)).center.values).T),axis=0))[0],
        density=np.array(np.abs(np.diff(np.sort(abs(np.matrix(Introns_info.query('Scaffold=='+str(scaffold)).center.values)-np.matrix(Peaks.query('Scaffold=='+str(scaffold)).center.values).T),axis=0)[0:2,:],axis=0)))[0]
    )


In [13]:
# drop missing values
Introns_info=Introns_info.dropna()
# select features
Introns_info=Introns_info[['Sense','Strong_donor','Strong_acceptor','Gene_ID','Scaffold','Intron_start_Mac51','Intron_stop_Mac51','WT_Spl','WT_Ret','WT_AS','NMD_Spl', 'NMD_Ret', 'NMD_AS','RPKM_WT','distance_center','nextExon_end_Mac51','nextExon_start_Mac51','prevExon_end_Mac51','prevExon_start_Mac51','Intron_position','Transcript_size','Transcript_start_Mac51','Transcript_end_Mac51','density','Donor','Acceptor','PTC_inducing','Count','old_Gene_ID','X3n_class']].reset_index(drop=True)
# convert features into integers
Introns_info[['nextExon_end_Mac51','nextExon_start_Mac51','prevExon_end_Mac51','prevExon_start_Mac51','Transcript_start_Mac51','Transcript_end_Mac51','Donor','Acceptor','Count']]=Introns_info[['nextExon_end_Mac51','nextExon_start_Mac51','prevExon_end_Mac51','prevExon_start_Mac51','Transcript_start_Mac51','Transcript_end_Mac51','Donor','Acceptor','Count']].astype(int)


In [14]:
# load genome sequece
fasta_dir='/Volumes/Storage3/Paramecium.FromChunlong/Data/Genome/mac_51/Scaffold/'
sequences=dict()
fasta_files=['20120207.scaffold51_'+str(i)+'.fa' for i in list(range(1,167))+[556]]
for i in list(range(1,167))+[556]:
    f = open(fasta_dir+'20120207.scaffold51_'+str(i)+'.fa','r')
    lines=f.readlines()[1:]
    f.close()
    sequences[i]=''.join([l.strip('\n') for l in lines])

In [15]:
# calculate GC content of the intron of interest
def Base_coup_content(sequences,Introns_info,i,what=['intron','prev_exon','next_exon','transcriptional_unit']):
    
    if what == 'intron':
        intron=sequences[Introns_info.Scaffold.iloc[i]][(Introns_info.Intron_start_Mac51.iloc[i]+1):Introns_info.Intron_stop_Mac51.iloc[i]-2]
    elif what == 'prev_exon':
        intron=sequences[Introns_info.Scaffold.iloc[i]][(Introns_info.prevExon_start_Mac51.iloc[i]-1):Introns_info.prevExon_end_Mac51.iloc[i]]
    elif what == 'next_exon':
        intron=sequences[Introns_info.Scaffold.iloc[i]][(Introns_info.nextExon_start_Mac51.iloc[i]-1):Introns_info.nextExon_end_Mac51.iloc[i]]
    elif what == 'transcriptional_unit':
        intron=sequences[Introns_info.Scaffold.iloc[i]][(Introns_info.Transcript_start_Mac51.iloc[i]-1):Introns_info.Transcript_end_Mac51.iloc[i]]

        
    def complementary(Sequence,sense):
        if sense == -1:
            Complementary={'A':'T','T':'A','C':'G','G':'C','N':'N'}
            s=[Complementary[s] for s in Sequence]
            s=''.join(s)
            return s
        else:
            return Sequence
    
    intron=complementary(intron,Introns_info.Sense.iloc[i])
    
    A=100*(intron.count('A'))/len(intron)
    T=100*(intron.count('T'))/len(intron)
    G=100*(intron.count('G'))/len(intron)
    C=100*(intron.count('C'))/len(intron)
    GC=100*(intron.count('C')+intron.count('G'))/len(intron)
    TC=100*(intron.count('C')+intron.count('T'))/len(intron)
    TG=100*(intron.count('T')+intron.count('G'))/len(intron)
    AT=100*(intron.count('A')+intron.count('T'))/len(intron)
    AC=100*(intron.count('C')+intron.count('A'))/len(intron)
    AG=100*(intron.count('A')+intron.count('G'))/len(intron)
    return (A,T,G,C,GC,TC,TG,AT,AC,AG)



Introns_info[['A','Ts','G','C','GC','TC','TG','AT','AC','AG']]=pd.DataFrame([Base_coup_content(sequences,Introns_info,i,'intron') for i in range(len(Introns_info.index))])
Introns_info[['A_nextExon','T_nextExon','G_nextExon','C_nextExon','GC_nextExon','TC_nextExon','TG_nextExon','AT_nextExon','AC_nextExon','AG_nextExon']]=pd.DataFrame([Base_coup_content(sequences,Introns_info,i,'next_exon') for i in range(len(Introns_info.index))])
Introns_info[['A_prevExon','T_prevExon','G_prevExon','C_prevExon','GC_prevExon','TC_prevExon','TG_prevExon','AT_prevExon','AC_prevExon','AG_prevExon']]=pd.DataFrame([Base_coup_content(sequences,Introns_info,i,'prev_exon') for i in range(len(Introns_info.index))])
Introns_info[['A_Transcrip','T_Transcrip','G_Transcrip','C_Transcrip','GC_Transcrip','TC_Transcrip','TG_Transcrip','AT_Transcrip','AC_Transcrip','AG_Transcrip']]=pd.DataFrame([Base_coup_content(sequences,Introns_info,i,'transcriptional_unit') for i in range(len(Introns_info.index))])


In [16]:
Introns_info=Introns_info.assign(DeltaGC=Introns_info.GC-(Introns_info.GC_nextExon+Introns_info.GC_prevExon)/2)

In [17]:
# calculate sizes of enxons
Introns_info['nextExon_Size']=1+Introns_info.nextExon_end_Mac51-Introns_info.nextExon_start_Mac51
Introns_info['prevExon_Size']=1+Introns_info.prevExon_end_Mac51-Introns_info.prevExon_start_Mac51


In [18]:
# free energy secondary structure
def DeltaG(sequences,Introns_info,i,size=50):
    
    def transcribe(Sequence,sense):
        if sense==1:
            s=str.replace(Sequence,'T','U')
        elif sense == -1:
            Complementary={'A':'U','T':'A','C':'G','G':'C','N':'N'}
            s=[Complementary[s] for s in Sequence]
            s.reverse()
            s=''.join(s)
        return s
    
    seq_intron=sequences[Introns_info.Scaffold.iloc[i]][(Introns_info.Intron_start_Mac51.iloc[i]-1-size):Introns_info.Intron_stop_Mac51.iloc[i]+size]
    seq_intron=transcribe(seq_intron,Introns_info.Sense.iloc[i])


        
    Before=RNA.fold(seq_intron[:size])[1]
    With=RNA.fold(seq_intron[:-size])[1]
    All=RNA.fold(seq_intron)[1]
    return (Before,With,All)

Introns_info[['DeltaG_50b','DeltaG_50b_int','DeltaG_50b_int_50a']]=pd.DataFrame([DeltaG(sequences,Introns_info,i) for i in range(len(Introns_info.index))])

In [19]:
#load nucleosome scores
data={
'D1':np.load('/Users/sgnan/Desktop/Projects/paramecium/tracks/D1_nucleosome_score.npy',allow_pickle='TRUE').item(),
'D2': np.load('/Users/sgnan/Desktop/Projects/paramecium/tracks/D2_nucleosome_score.npy',allow_pickle='TRUE').item(),
'S41': np.load('/Users/sgnan/Desktop/Projects/paramecium/tracks/S41_nucleosome_score.npy',allow_pickle='TRUE').item(),
'S32new': np.load('/Users/sgnan/Desktop/Projects/paramecium/tracks/S32new_nucleosome_score.npy',allow_pickle='TRUE').item()
}

In [20]:
# Average and correct normalise control
data['D']={scaffold:0.01+(data['D1'][scaffold]+data['D2'][scaffold])/2 for scaffold in data['D1'].keys()}
data['S']={scaffold:0.01+(data['S41'][scaffold]+data['S32new'][scaffold])/2 for scaffold in data['S32new'].keys()}

data['Norm']={scaffold:(data['S'][scaffold]/data['D'][scaffold]) for scaffold in data['S'].keys()}
# set nan and Inf to zero
for scaffold in data['Norm'].keys():
    data['Norm'][scaffold][np.isfinite(data['Norm'][scaffold])==False]=0

In [21]:
# calculate median nucleosome score over introns
def MNasesignal(sample_Track,Introns_info,i):
    intron=np.median(sample_Track[Introns_info.Scaffold.iloc[i]][(Introns_info.Intron_start_Mac51.iloc[i]):Introns_info.Intron_stop_Mac51.iloc[i]+1])
    return intron
# calculate median nucleosome score over following exon
def MNasesignal_next(sample_Track,Introns_info,i):
    intron=np.median(sample_Track[Introns_info.Scaffold.iloc[i]][(Introns_info.nextExon_start_Mac51.iloc[i]):Introns_info.nextExon_end_Mac51.iloc[i]+1])
    return intron
# calculate median nucleosome score over previous exon
def MNasesignal_prev(sample_Track,Introns_info,i):
    intron=np.median(sample_Track[Introns_info.Scaffold.iloc[i]][(Introns_info.prevExon_start_Mac51.iloc[i]):Introns_info.prevExon_end_Mac51.iloc[i]+1])
    return intron

Introns_info=Introns_info.assign(
MNase=[MNasesignal(data['Norm'],Introns_info,i) for i in range(len(Introns_info.index))],
MNase_nextExon=[MNasesignal_next(data['Norm'],Introns_info,i) for i in range(len(Introns_info.index))],
MNase_prevExon=[MNasesignal_prev(data['Norm'],Introns_info,i) for i in range(len(Introns_info.index))]
)

In [22]:
#calculate splicing efficiency and length of introns
Introns_info=Introns_info.assign(
Splice_eff_WT= (Introns_info.WT_Spl)/(Introns_info.WT_Spl+Introns_info.WT_Ret+Introns_info.WT_AS),
Splice_eff_NMD= (Introns_info.NMD_Spl)/(Introns_info.NMD_Spl+Introns_info.NMD_Ret+Introns_info.NMD_AS),
len_intron= 1+Introns_info.Intron_stop_Mac51-Introns_info.Intron_start_Mac51
).dropna()



In [23]:
#add PTC absolute position from TSS and TTS
Introns_info['distance_from_TSS']=(np.abs(Introns_info.apply(lambda x: x['Transcript_start_Mac51'] if x['Sense']==1 else x['Transcript_end_Mac51'],1).values-(Introns_info.Intron_start_Mac51.values+Introns_info.Intron_stop_Mac51.values)/2))
Introns_info['distance_from_TTS']=(np.abs(Introns_info.apply(lambda x: x['Transcript_start_Mac51'] if x['Sense']==-1 else x['Transcript_end_Mac51'],1).values-(Introns_info.Intron_start_Mac51.values+Introns_info.Intron_stop_Mac51.values)/2))


In [24]:
# save to file 
Introns_info.to_csv('Intron_infos.tsv',sep='\t',index=False)